In [4]:
import pandas as pd
import kmapper as km
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('horse.csv')

# 提取三维坐标
points = df[['x', 'y', 'z']].values

In [7]:
# 创建Kepler Mapper对象
mapper = km.KeplerMapper()

# 使用PCA对数据进行初步的降维操作
pca = PCA(n_components=2)
projected_data = pca.fit_transform(points)


In [8]:
# 用PCA映射得到的2D数据
lens = projected_data[:, 0]  # 使用第一主成分作为 lens

# 应用DBSCAN聚类
graph = mapper.map(lens,
                   points,
                   clusterer=DBSCAN(eps=0.5, min_samples=5),
                   cover=km.Cover(n_cubes=10, perc_overlap=0.3))


In [9]:
# 绘制结果
mapper.visualize(graph, path_html="mapper_output.html")

'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Kepler Mapper | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-size: 13px;\n  font-weigh

In [14]:
from sklearn.preprocessing import StandardScaler
# 标准化数据
scaler = StandardScaler()
points_scaled = scaler.fit_transform(points)

In [15]:
import numpy as np
# 使用欧几里得距离作为Lens
lens = np.linalg.norm(points_scaled, axis=1)

In [16]:
mapper = km.KeplerMapper()

# 覆盖参数：调整 n_cubes 和 perc_overlap
graph = mapper.map(lens, 
                   points_scaled, 
                   clusterer=DBSCAN(eps=0.5, min_samples=5), 
                   cover=km.Cover(n_cubes=15, perc_overlap=0.5))

In [18]:
mapper.visualize(graph, path_html="mapper_output.html")

'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Kepler Mapper | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-size: 13px;\n  font-weigh

In [25]:
import plotly.graph_objects as go

# 提取节点和边的数据
nodes = {}
edges = []

# 将节点转换为字典，键为节点ID，值为节点信息
for node_id, members in graph['nodes'].items():
    # 将节点信息（如成员数量和位置）存储在字典中
    nodes[node_id] = {
        'id': node_id,
        'members': len(members),
        'position': np.mean(points_scaled[members], axis=0)  # 节点位置设为该节点内成员的平均位置
    }

# 收集边的信息
for node_a, linked_nodes in graph['links'].items():
    for node_b in linked_nodes:
        edges.append((node_a, node_b))

# 构建节点的坐标
x_nodes = [node['position'][0] for node in nodes.values()]
y_nodes = [node['position'][1] for node in nodes.values()]
z_nodes = [node['position'][2] for node in nodes.values()]

# 构建边的坐标
edge_x = []
edge_y = []
edge_z = []

for edge in edges:
    # 从字典中获取每个节点的位置
    x0, y0, z0 = nodes[edge[0]]['position']
    x1, y1, z1 = nodes[edge[1]]['position']
    
    # 构建边的坐标，用 None 断开线段
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

# 创建 Plotly 图形
fig = go.Figure()

# 添加边线
fig.add_trace(go.Scatter3d(x=edge_x, y=edge_y, z=edge_z,
                           mode='lines',
                           line=dict(color='grey', width=2),
                           hoverinfo='none'))

# 添加节点
fig.add_trace(go.Scatter3d(x=x_nodes, y=y_nodes, z=z_nodes,
                           mode='markers',
                           marker=dict(size=[10 + node['members']*2 for node in nodes.values()],  # 节点大小反映成员数量
                                       color='blue',
                                       opacity=0.8),
                           text=[f"Node {node['id']}<br>Members: {node['members']}" for node in nodes.values()],
                           hoverinfo='text'))

# 设置布局
fig.update_layout(title='Interactive Mapper Graph',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'),
                  margin=dict(l=0, r=0, b=0, t=50))

# 显示图形
fig.show()
